In [593]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.lines import Line2D

# Carga de tablas 

In [594]:
# Load the XLSX files into Pandas dataframes
Inventario = pd.read_excel('Inventarios/20-04.xlsx', skiprows=2)
Ventas = pd.read_excel('Ventas/Ventas.xlsx', skiprows=2)
Utilidad = pd.read_excel('Ventas/Utilidad.xlsx', skiprows=2)
Cartera = pd.read_excel('Cartera/20-04.xlsx', skiprows=2)
Egresos = pd.read_excel('Egresos/Egresos.xlsx', skiprows=2)
Egresos_ = pd.read_excel('Egresos/Egresos_.xlsx', skiprows=2)
Pendiente = pd.read_excel('Suministro/Pendiente.xlsx', skiprows=2)
IQChemicals = pd.read_excel('IQChemicals/Otros.xlsx', skiprows=2)
IQChemicals_mp = pd.read_excel('IQChemicals/Materia_prima.xlsx', skiprows=2)
CxP = pd.read_excel('CxP/CxP.xlsx', skiprows=2)

# Tabla auxiliar para traducir unidades en costo
Pres_a_vol = pd.read_excel('Categorias/Presentacion_a_volumen.xlsx')
Item_a_producto = pd.read_excel('Categorias/Item_a_producto.xlsx')
Costos_unitarios = pd.read_excel('Costo_unitario/Costo.xlsx',skiprows=2)
Producto = pd.read_excel('Categorias/Producto.xlsx')


#La hoja de inventario IqChemicals, tiene dos tablas. Para el análisis, se consolidan en una:

In [595]:
IQChemicals_mp_otros = pd.concat([IQChemicals,IQChemicals_mp])

Cartera, se desglosa en Vencida y corriente

In [596]:
Cartera['ESTADO'] = np.where(Cartera['ANTIGUEDAD'] == '  VENCIDA', 'VENCIDA', 'CORRIENTE')

Cartera corriente

In [597]:
Cartera_corriente = Cartera[Cartera['ESTADO'] == 'CORRIENTE']
Cartera_vencida = Cartera[Cartera['ESTADO'] == 'VENCIDA']

In [598]:
#Inventario, incluir columna de ABC

Item_a_ABC = Item_a_producto.groupby(['Item','ABC']).size().reset_index()
Inventario = Inventario.rename(columns = {'Número de artículo':'Item'})
Inventario = Inventario.merge(Item_a_ABC, how = 'left', on = 'Item')

In [599]:
#Ventas, incluir columna de categoría

Producto = Producto.groupby(['Producto_Ranking','Respaldo categorías']).size().reset_index()

Producto['Producto_Ranking'] = Producto['Producto_Ranking'].str.upper().str[3:]

Ventas['Producto_Ranking'] = Ventas['Producto_Ranking'].str[3:]
Utilidad['Producto_Ranking'] = Utilidad['Producto_Ranking'].str[3:]


Limpieza de homógos
JUNIPERUS no es igual que JUNÍPERUS... LPG 456 REACTMAXX, no es igual que LPG 456 -REACTMAXX...
La siguiente sección incluye algunos homólogos para que todos las filas crucen:

In [600]:
# Listado de productos que no cruzaron originalmente (Limpieza de homólogos)
Homologos_adicionales = pd.DataFrame({
    'Producto_Ranking': ['JUNIPERUS STRESSOFF', 'SPROUD MAX', 'LPG 456 - REACTMAXX', 'AGROPLEXBMO STRESSOFF', 
                'KIT-CALIDAD-IQC', 'KIT-ESTRUCTURAL-IQC', 'OTROS'],
    'Respaldo categorías': ['Bioestimulante', 'Cito', 'Cito', 'Bioestimulante', 
                 'Bioestimulante', 'Cito', 'Bioestimulante']
})

# append the corrected mapping to the producto dataframe
Producto = pd.concat([Producto,Homologos_adicionales], ignore_index=True)



In [601]:
# Inclusión de columna categorías (Cito, Raíz, Bioestimulante...) en hoja de Ventas e Utilidad
Ventas = Ventas.merge(Producto, on = 'Producto_Ranking', how = 'left')
Utilidad = Utilidad.merge(Producto, on = 'Producto_Ranking', how = 'left')

In [602]:
#Pendiente, incluir columna de categoría
Item_a_categoría = Item_a_producto.groupby(['Item','Categoría']).size().reset_index()
Pendiente = Pendiente.rename(columns = {'ITEMNMBR':'Item'})
Pendiente = Pendiente.merge(Item_a_categoría, how = 'left', on = 'Item')

In [603]:
#IQChemicals_homologación de categorías

# Create a new column with updated values
IQChemicals_mp_otros['ALMACEN_'] = np.where(IQChemicals_mp_otros['ALMACEN'].isin(['ALMACEN DE PRODUCTO TERMINADO CHEMICALS', 'PRODUCTO TERMINADO TLAXCALA']),
                                              'PRODUCTO TERMINADO',
                                              IQChemicals_mp_otros['ALMACEN'])

# Indice de tablas

In [604]:
#df - Ventas
#df1 - Utilidad
#df2 - Inventario
#df3 - Cartera_vencida
#df4 - Egresos
#df5 - Pendiente
#df6 - Costo
#df7 - Inventario IQC
#df8 - Cartera_corriente
#df9 - CxP
#df10 - Solicitado

# Homologación de nombres de columnas

In [605]:
#Ventas 2023, valores superiores a cero, renombrar columnas
Ventas = Ventas.rename(columns={
       'Empresa Jerarquía - Zona Ranking_':'Zona',
       'Empresa Jerarquía - RM/ REIQ_Ranking':'RM',
       'Empresa Jerarquía - Cultivo_ranking':'Cultivo',
       'ID_PRODUCTO':'Item', 
       "'Base'[Mes]'":"Mes",
       'Ver selección':'Ventas',
       'Respaldo categorías':'Nivel 2'
                        }
               )
Ventas = Ventas[Ventas['Ventas'] > 0]

In [606]:
#Utilidad 2023, renombrar columnas
Utilidad = Utilidad.rename(columns={
       'Empresa Jerarquía - Zona Ranking_':'Zona',
       'Empresa Jerarquía - RM/ REIQ_Ranking':'RM',
       'Empresa Jerarquía - Cultivo_ranking':'Cultivo',
       'ID_PRODUCTO':'Item', 
       "'Base'[Mes]'":"Mes",
       'Ver selección':'Utilidad',
       'Respaldo categorías':'Nivel 2'
                        }
               )
#Filtros de Utilidad 2023, valores superiores a cero
Utilidad = Utilidad[Utilidad['Utilidad'] > 0]

In [607]:

#Inventario, renombrar columnas
Inventario = Inventario.rename(columns = {
    'EMPRESA':'Empresa',
    'VALOR INVENTARIO':'Inventario_valor',
    'Número de artículo':'Item',
    'ABC':'Nivel 2'
    
                                        }
                              )
#Filtros de Inventario, valores superiores a uno, 

Inventario = Inventario[Inventario['Inventario_valor'] > 0]

In [608]:
#Cartera vencida, renombrar columnas
Cartera_vencida = Cartera_vencida.rename(columns = {
    "'VW_CxC_COM_V4'[EMPRESA]":"Empresa",
    "Sum of TOTAL":"Cartera_vencida",
    "ANTIGUEDAD":"Nivel 2"
                                        }
                              )
#Filtros de Cartera, valores superiores a uno, 

Cartera_vencida = Cartera_vencida[Cartera_vencida['Cartera_vencida'] > 0]

In [609]:
#Cartera_corriente, renombrar columnas
Cartera_corriente = Cartera_corriente.rename(columns = {
    "'VW_CxC_COM_V4'[EMPRESA]":"Empresa",
    "Sum of TOTAL":"Cartera_corriente",
    "ANTIGUEDAD":"Nivel 2"
                                        }
                              )
#Filtros de Cartera_corriente, valores superiores a uno, 

Cartera_corriente = Cartera_corriente[Cartera_corriente['Cartera_corriente'] > 0]

In [610]:
#Egresos, renombrar columnas
Egresos_ = Egresos_.rename(columns = {
    "NIVEL4":"Empresa",
    "Sum of MONTO":"Egresos_monto",
    "NIVEL2":"Nivel 2"
                                        }
                              )
#Filtros de Egresos, valores superiores a uno, 

Egresos_ = Egresos_[Egresos_['Egresos_monto'] > 0]
Egresos_ = Egresos_[Egresos_['FUENTE'] == "REAL"]

In [611]:
#Pendiente, renombrar columnas
Pendiente = Pendiente.rename(columns = {
    "EMPRESA":"Empresa",
    "Sum of PENDIENTE":"Unidades",
    "PRESENTACION":"Presentación",
    "Categoría":"Nivel 2",
    "Sum of SOLICITADO":"Unidades_"
                                        }
                              )
#Filtros de Egresos, valores superiores a uno, 


Formulaciones_a_excluir = ['AGE-21', 'AABC-22', 'ARP4-20', 'AC GL-22']
Pendiente = Pendiente[~Pendiente['FORMULACION'].isin(Formulaciones_a_excluir)]

In [612]:
#Pres_a_vol, NO requiere renombrar columnas
#Pres_a_vol.columns 

Listado de cols:
'Presentación', 'Vol aux'

In [613]:
#Costos_unitarios, renombrar columnas
Costos_unitarios = Costos_unitarios.rename(columns = {
    "PRESENTACION":"Presentación"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

Costos_unitarios = Costos_unitarios[Costos_unitarios['Sum of K/L'] > 0]

In [614]:
#Costos_unitarios, renombrar columnas
IQChemicals_mp_otros = IQChemicals_mp_otros.rename(columns = {
    "EMPRESA":"Empresa",
    "VALOR INVENTARIO":"Inventario_IQC_Valor",
    "ALMACEN_":"Nivel 2"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

IQChemicals_mp_otros = IQChemicals_mp_otros[IQChemicals_mp_otros['Inventario_IQC_Valor'] > 0]

In [615]:
CxP.columns

Index(['GPO', 'EMPRESA', 'RUBRO', 'PROVEEDOR', 'TOTAL', 'TIPOS'], dtype='object')

In [616]:
#CxP, renombrar columnas
CxP = CxP.rename(columns = {
    "EMPRESA":"Empresa",
    "TOTAL":"CxP",
    "RUBRO":"Nivel 2"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

CxP = CxP[CxP['CxP'] > 0]

# Tabla auxiliar para pasar de unidades a costo

In [617]:
#Costo por formulación y presentación
Costos_unitarios = Costos_unitarios.groupby(['FORMULACION','Presentación'])['COSTOUNITREAL.'].mean().reset_index()

In [618]:
# Costos NO incluidos en tablero.
new_data = pd.DataFrame({
    'FORMULACION': ['APLG-21', 'BRXCO-21', 'AV5G-22', 'AV5G-22', 'BTR-22'],
    'Presentación': ['1X200', '12X1', '1X20', '4X5', '4X5'],
    'COSTOUNITREAL.': [20, 70, 80, 80, 200]
})

# Se consolida tabla para que no haya valores en blanco al cruzar con Pendiente
Costos_unitarios = pd.concat([Costos_unitarios, new_data], ignore_index=True)


In [619]:
#Factor de volumenn por presentación
Pres_a_vol_ = Pres_a_vol.groupby(['Presentación'])['Vol aux'].mean().reset_index()

In [620]:
#Unidades a volumen
Pendiente = Pendiente.merge(Pres_a_vol_, on = 'Presentación', how = 'left')
Pendiente['Volumen'] = Pendiente['Vol aux']*Pendiente['Unidades']
Pendiente['Volumen_'] = Pendiente['Vol aux']*Pendiente['Unidades_']
# Volumen a costo
Pendiente = Pendiente.merge(Costos_unitarios, on = ['FORMULACION', 'Presentación'], how = 'left')
Pendiente['Pendiente_valor'] = Pendiente['Volumen']*Pendiente['COSTOUNITREAL.']
Pendiente['Pendiente_solicitado'] = Pendiente['Volumen_']*Pendiente['COSTOUNITREAL.']
Pendiente.to_csv('Pend_vs_Sol.csv')

# Agrupar componentes financieros por empresa

In [621]:
# group Ventas por empresa
df = Ventas.groupby(['Empresa','Nivel 2'])['Ventas'].sum().reset_index()
df['Fuente'] = "Ventas"

In [622]:
# group Utilidad por empresa
df1 = Utilidad.groupby(['Empresa','Nivel 2'])['Utilidad'].sum().reset_index()
df1['Fuente'] = "Utilidad"

In [623]:
# group Inventario por empresa
df2 = Inventario.groupby(['Empresa','Nivel 2'])['Inventario_valor'].sum().reset_index()
df2['Fuente'] = "Inventario_valor"
df2 = df2.rename(columns = {'Inventario_valor':'Valor'})

In [624]:
Cartera_vencida.head()

,GPO,Empresa,ZONA,CLIENTE,CATEGORIA,Cartera_vencida,Nivel 2,ESTADO
7,GRUPO,INT,"Ecu,Col",AGRICOLAS ANTIOQUIA SAS,AdStrong,2.044621e+06,VENCIDA,VENCIDA
8,GRUPO,CAC,Zona 2,AGRICOLA PISCIS SA,AdStrong,1.764476e+06,VENCIDA,VENCIDA
9,GRUPO,IQCHEM,REIQ,CORPORATIVO TXAO,ReactMaxx,1.320918e+06,VENCIDA,VENCIDA
11,GRUPO,IQCHEM,REIQ,CORPORATIVO TXAO,StressOff,1.305532e+06,VENCIDA,VENCIDA
17,GRUPO,CAC,Zona 2,AGROASA,ReactMaxx,1.113007e+06,VENCIDA,VENCIDA


In [625]:
# group Cartera por empresa
df3 = Cartera_vencida.groupby(['Empresa','Nivel 2'])['Cartera_vencida'].sum().reset_index()
df3['Fuente'] = "Cartera_vencida"
df3 = df3.rename(columns = {'Cartera_vencida':'Valor'})

In [626]:
# group Egresos por empresa
df4 = Egresos_.groupby(['Empresa','Nivel 2'])['Egresos_monto'].sum().reset_index()
df4['Fuente'] = "Egresos_monto"
df4 = df4.rename(columns = {'Egresos_monto':'Valor'})

In [627]:
# group Pendiente por empresa
df5 = Pendiente.groupby(['Empresa','Nivel 2'])['Pendiente_valor'].sum().reset_index()
df5['Fuente'] = "Pendiente"
df5 = df5.rename(columns = {'Pendiente_valor':'Valor'})

In [628]:
# group Solicitado por empresa
df10 = Pendiente.groupby(['Empresa','Nivel 2'])['Pendiente_solicitado'].sum().reset_index()
df10['Fuente'] = "Solicitado"
df10 = df10.rename(columns = {'Pendiente_solicitado':'Valor'})

# Costo

In [629]:
# Combinar Utilidad y Ventas
df_merged = pd.merge(df, df1, on=['Empresa', 'Nivel 2'], how='left')


# Calcular Costo como diferencia de Ventas e Utilidad
df_merged['Costo'] = df_merged['Ventas'] - df_merged['Utilidad']

# Agrupar Costo por Empresa
df6 = df_merged.groupby(['Empresa','Nivel 2'])['Costo'].sum().reset_index()
df6['Fuente'] = "Costo"
df6 = df6.rename(columns = {'Costo':'Valor'})

Agregar columna Fuente en Ventas e Utilidad
Se agregó después porque esas columnas iban a utilizarse para la tabla de costos

In [630]:
df_ = df.rename(columns = {'Ventas':'Valor'})
df1_ = df1.rename(columns = {'Utilidad':'Valor'})

# Inventario IQC valor

In [631]:
# group Inventario IQC por empresa
df7 = IQChemicals_mp_otros.groupby(['Empresa','Nivel 2'])['Inventario_IQC_Valor'].sum().reset_index()
df7['Fuente'] = "Inventario_IQC_Valor"
df7 = df7.rename(columns = {'Inventario_IQC_Valor':'Valor'})

In [632]:
# group Cartera corriente
df8 = Cartera_corriente.groupby(['Empresa','Nivel 2'])['Cartera_corriente'].sum().reset_index()
df8['Fuente'] = "Cartera_corriente"
df8 = df8.rename(columns = {'Cartera_corriente':'Valor'})


In [633]:
# group CxP
df9 = CxP.groupby(['Empresa','Nivel 2'])['CxP'].sum().reset_index()
df9['Fuente'] = "CxP"
df9 = df9.rename(columns = {'CxP':'Valor'})

# Componentes financieros consolidados

In [634]:
dfs = [df_, df1_, df2, df3, df4, df5, df6, df7, df8, df9, df10]
cols = ['Empresa', 'Nivel 2', 'Fuente', 'Valor']
df_merged = pd.concat(dfs, axis=0)



# Categorías de componentes

In [635]:
# Define the conditions and categories
conditions = [df_merged['Fuente'].isin(['Ventas', 'Utilidad']),
    df_merged['Fuente'].isin(['Egresos_monto', 'Costo', 'CxP']),
    df_merged['Fuente'].isin(['Cartera_vencida', 'Cartera_corriente','Inventario_valor', 'Pendiente','Inventario_IQC_Valor']),
    df_merged['Fuente'].isin(['Solicitado'])
]

categories = ['01 Ingresos','02 Egresos','03 Inventario y Cartera','Sol']

# Apply the conditions and categories to create a new "Category" column
df_merged['Categoria'] = np.select(conditions, categories)

# Exportar tabla para uso en tableros o tablas dinámicas

In [636]:
df_merged.to_csv('Componentes_financieros_nivel_2.csv')

# Componentes consolidados

In [637]:
# group by "Component" and "Category", format the "Value" column as currency, and sum the values
df_merged['Valor (M)'] = df_merged['Valor'].apply(lambda x: x / 1000000)

# group by "Component" and "Category", format the "Value" column as currency, and sum the values
Componentes = df_merged.groupby(['Categoria', 'Fuente'])['Valor (M)'].apply(lambda x: x.sum()).reset_index()

# pivot the table to put the "Category" values as columns
Componentes_ = Componentes.pivot(index='Fuente', columns='Categoria', values='Valor (M)')
Componentes_ = Componentes_.fillna(0)

# format the currency columns
for col in Componentes_.columns:
    Componentes_[col] = Componentes_[col].apply(lambda x: "${:,.1f}".format(x))

# print the result
print(Componentes_)



Categoria            01 Ingresos 02 Egresos 03 Inventario y Cartera    Sol
Fuente                                                                    
Cartera_corriente           $0.0       $0.0                   $65.1   $0.0
Cartera_vencida             $0.0       $0.0                   $15.4   $0.0
Costo                       $0.0      $38.5                    $0.0   $0.0
CxP                         $0.0      $45.3                    $0.0   $0.0
Egresos_monto               $0.0      $48.7                    $0.0   $0.0
Inventario_IQC_Valor        $0.0       $0.0                   $23.7   $0.0
Inventario_valor            $0.0       $0.0                   $24.7   $0.0
Pendiente                   $0.0       $0.0                    $3.7   $0.0
Solicitado                  $0.0       $0.0                    $0.0  $38.3
Utilidad                   $75.7       $0.0                    $0.0   $0.0
Ventas                    $114.2       $0.0                    $0.0   $0.0


In [638]:
# create a dictionary to map the old values to the new values
fuente_dict = {'Ventas': 'Vent', 'Utilidad': 'Uti', 'Inventario_valor': 'Inv', 'Cartera_vencida': 'Cart Ok', 'Egresos_monto': 'Gasto', 'Pendiente': 'PPE', 'Costo': 'Cost', 'Inventario_IQC_Valor': 'Inv IQC', 'Cartera_corriente': 'Cart Ok', 'CxP': 'CxP', 'Solicitado':'Sol'}
fuente_dict_ = {'01 Ingresos': 'Ut y V', '02 Egresos': 'Egr', '03 Inventario y Cartera': 'Inv y Cart', 'Sol':'S'}

# replace the old values with the new values using the dictionary
df_merged['Fuente'] = df_merged['Fuente'].replace(fuente_dict)
df_merged['Categoria'] = df_merged['Categoria'].replace(fuente_dict_)

Detalle = df_merged.groupby(['Fuente', 'Nivel 2','Categoria'])['Valor (M)'].apply(lambda x: x.sum()).reset_index()

# pivot the table to put the "Category" values as columns
Detalle_ = Detalle.pivot(index=['Fuente','Nivel 2'], columns='Categoria', values='Valor (M)')

Detalle_ = Detalle_.fillna(0)
# format the currency columns
for col in Detalle_.columns:
    Detalle_[col] = Detalle_[col].apply(lambda x: "${:,.1f}".format(x))

# print the result
print(Detalle_.head(60))
Detalle_.to_csv('Resumen_comp_financieros.csv')

Categoria                              Egr Inv y Cart      S Ut y V
Fuente  Nivel 2                                                    
Cart Ok   VENCIDA                     $0.0      $15.4   $0.0   $0.0
         EN EL MES                    $0.0      $30.5   $0.0   $0.0
        A 30 DIAS                     $0.0      $26.3   $0.0   $0.0
        A 60 DIAS                     $0.0       $6.7   $0.0   $0.0
        A 90 +                        $0.0       $1.6   $0.0   $0.0
Cost    Bioestimulante                $7.8       $0.0   $0.0   $0.0
        Cito                          $8.2       $0.0   $0.0   $0.0
        Coadyuvante                   $3.2       $0.0   $0.0   $0.0
        GA                            $7.9       $0.0   $0.0   $0.0
        Maquila                       $0.1       $0.0   $0.0   $0.0
        Nutrición                     $2.5       $0.0   $0.0   $0.0
        Otro                          $0.0       $0.0   $0.0   $0.0
        Raíz/ suelo                   $8.8      

In [639]:
# group by "Component" and "Category", format the "Value" column as currency, and sum the values
Componentes_p_Emp = df_merged.groupby(['Fuente', 'Empresa'])['Valor (M)'].apply(lambda x: x.sum()).reset_index()

# pivot the table to put the "Category" values as columns
Componentes_p_Emp_ = Componentes_p_Emp.pivot(index='Empresa', columns='Fuente', values='Valor (M)')
Componentes_p_Emp_ = Componentes_p_Emp_.fillna(0)

# format the currency columns
for col in Componentes_p_Emp_.columns:
    Componentes_p_Emp_[col] = Componentes_p_Emp_[col].apply(lambda x: "${:,.1f}".format(x))

# print the result
Componentes_p_Emp_ = Componentes_p_Emp_.fillna(0)
print(Componentes_p_Emp_)


Fuente   Cart Ok   Cost    CxP Gasto   Inv Inv IQC   PPE    Sol    Uti   Vent
Empresa                                                                      
ASESORIA    $0.0   $0.0   $0.0  $0.0  $0.0    $0.0  $0.0   $0.0   $0.0   $0.0
CAC        $18.5   $7.8   $1.5  $5.4  $1.9    $0.0  $0.0   $0.8   $8.2  $16.0
CHEMI       $0.0   $0.0  $20.3  $5.0  $0.0   $23.7  $0.0   $0.0   $0.0   $0.0
ENFOKT      $0.0   $0.0   $0.0  $0.0  $0.0    $0.0  $0.0   $0.0   $0.0   $0.0
INT         $5.0   $1.3   $1.0  $1.6  $0.1    $0.0  $0.0   $0.6   $2.4   $3.7
IQCHEM      $8.5   $2.2   $0.6  $3.9  $8.5    $0.0  $0.1   $1.2   $5.1   $7.3
LASA        $0.0   $0.0   $3.1  $8.0  $0.0    $0.0  $0.0   $0.0   $0.0   $0.0
MX         $41.8  $26.1   $3.2  $9.8  $9.7    $0.0  $3.6  $34.4  $54.7  $80.8
RETENUM     $0.0   $0.0  $10.3  $9.6  $0.0    $0.0  $0.0   $0.0   $0.0   $0.0
SACH        $0.0   $0.0   $3.2  $0.0  $0.0    $0.0  $0.0   $0.0   $0.0   $0.0
SUD         $6.7   $1.0   $2.1  $5.4  $4.4    $0.0  $0.0   $1.3 

In [640]:
df_merged.head()


,Empresa,Nivel 2,Valor,Fuente,Categoria,Valor (M)
0,CAC,Cito,1836636.00,Vent,Ut y V,1.836636
1,CAC,Coadyuvante,727750.00,Vent,Ut y V,0.727750
2,CAC,GA,12545077.50,Vent,Ut y V,12.545077
3,CAC,Raíz/ suelo,907084.00,Vent,Ut y V,0.907084
4,INT,Bioestimulante,1537523.02,Vent,Ut y V,1.537523


In [641]:
df_merged.Empresa.unique()

array(['CAC', 'INT', 'IQCHEM', 'MX', 'SUD', 'CHEMI', 'LASA', 'RETENUM',
       'ASESORIA', 'ENFOKT', 'SACH'], dtype=object)

In [642]:
df_merged['Nivel 2'].unique()

array(['Cito', 'Coadyuvante', 'GA', 'Raíz/ suelo', 'Bioestimulante',
       'Nutrición', 'Otro', 'Maquila', ' A ', 'D', 'B', 'C', '  VENCIDA',
       'ADMINISTRACION', 'OFICINA Y EQUIPO DE TRABAJO', 'PERSONAL',
       'VARIABLES', 'FLETES Y REGISTROS', 'CONSIGNA', 'EMPAQUE',
       'EN PROCESO', 'EN TRANSITO', 'ETIQUETAS', 'INDIRECTOS',
       'MATERIA PRIMA', 'OBSOLETO', 'PRODUCTO TERMINADO',
       'TEMPORAL IQCHEM', ' EN EL MES', 'A 30 DIAS', 'A 90 +',
       'A 60 DIAS', 'GASTOS FIJOS', 'CONSUMIBLES', 'SERVICIOS',
       'INVERSIONES', 'PROMOCIONALES'], dtype=object)